In [2]:
import pandas as pd
import numpy as np
import openpyxl
import sqlite3
import os

In [3]:
# CONSTANTS
FILE_EXTENSION = ".txt"
FILEPATH = "./files"
TABLES_SEPARATER = '\n\n'
TABLE_DATA_SEPRATER = '\n'
TABLENAME_STARTING_DELIMITER = "*** "
TABLENAME_ENDING_DELIMTER = " ***"
COLUMNS_SEPARATER = '\t'
TABLE_DATA_SEPARATER = '\t'

TABLENAME_CLEANING_CHARACTERS = [(" ", "_"),("&", "_"),("-", "_")]

In [4]:
requiredFilesList = list()

with os.scandir(FILEPATH) as currentDirectory:
    for entry in currentDirectory:
        
        # Criteria for selecting the required files
        criteriaSet = set()
        criteriaSet.add(entry.name.endswith(FILE_EXTENSION))
        criteriaSet.add((entry.name.lower().find("before") > -1) or (entry.name.lower().find("after") > -1))
        if False not in criteriaSet:
            requiredFilesList.append(entry.name)

# Updating the file names to include the path
requiredFilesList = [FILEPATH + "/" + file for file in requiredFilesList]
requiredFilesList.sort()
requiredFilesList

['./files/test2After.txt', './files/test2Before.txt']

In [5]:
for entry in requiredFilesList:

    fileType = "after"
    if entry[:-len(FILE_EXTENSION)][-1] == "e": fileType = "before"
        
    comparisonList = list()
    dbName = entry[ : -len(FILE_EXTENSION) - len(fileType)]
    con = sqlite3.connect(dbName + '.db')
    cur = con.cursor()
    file=open(entry,"r")
    
    for table in file.read().split(TABLES_SEPARATER):
        
        tableInfo = table.split(TABLE_DATA_SEPRATER)
        if tableInfo != '' and len(tableInfo) > 1:
            
            tableName = tableInfo[0][len(TABLENAME_STARTING_DELIMITER) : -len(TABLENAME_ENDING_DELIMTER)]
            tableName = tableName + "_" + fileType
            for clean_char in TABLENAME_CLEANING_CHARACTERS:
                tableName = tableName.replace(clean_char[0], clean_char[1])
            
            tableColumns = list()
            for column in tableInfo[1].split(COLUMNS_SEPARATER):
                tableColumns.append(column.replace(".", "_"))
            tableColumns = tuple(tableColumns)
            
            tableCreationQuery = 'CREATE TABLE {table_name} {table_columns}'
            tableCreationQuery = tableCreationQuery.format(table_name = tableName, table_columns = tableColumns)
            try:
                cur.execute(tableCreationQuery)
            except sqlite3.OperationalError:
                pass
            
            for data in tableInfo[2:]:
                
                row_data = tuple(data.split(TABLE_DATA_SEPARATER))
                tableDataInsertionQuery = 'INSERT INTO {table_name} VALUES {table_data}'
                tableDataInsertionQuery = tableDataInsertionQuery.format(table_name = tableName, table_data = row_data)
                cur.execute(tableDataInsertionQuery)
            
            con.commit()             
                